In [44]:
# # На случай открытия в Google Colab
# !pip install lightgbm optuna scikit-learn pandas numpy scipy matplotlib seaborn imbalanced-learn 

# !pip install --upgrade scikit-learn      

In [45]:
# Основные библиотеки
import numpy as np
import pandas as pd
import warnings

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# Машинное обучение и модели
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

# Метрики и оценка
from sklearn.metrics import (
    classification_report, confusion_matrix,
    precision_score, recall_score, f1_score, fbeta_score
)

# Предобработка данных
from sklearn.preprocessing import (
    OneHotEncoder, KBinsDiscretizer, StandardScaler,
)
from sklearn.feature_extraction import FeatureHasher
from sklearn.compose import ColumnTransformer
from scipy.sparse import issparse
from imblearn.under_sampling import ClusterCentroids

# Разбиение данных и кросс-валидация
from sklearn.model_selection import (
    train_test_split, StratifiedKFold
)

# Оптимизация гиперпараметров
import optuna

warnings.filterwarnings('ignore')

In [24]:
# Извлечение данных из csv-таблиц
df_train = pd.read_csv('Task/df_train.csv')
df_test = pd.read_csv('Task/df_test.csv')

# Удаление дубликатов
df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()

y_train_np = df_train['target'].to_numpy()
y_test_np = df_test['target'].to_numpy()

df_train.drop(columns=['target'], inplace=True)
df_test.drop(columns=['target'], inplace=True)

In [25]:
# Кодирование категориальных признаков
categorical_cols = ['PaymentType', 'service']

for i,df in enumerate([df_train, df_test]):
    df.drop(columns=['user_id', 'CreatedDate', 'NmAge','number_of_ordered_items'], inplace=True)
    df['IsPaid'] = df['IsPaid'].map({False: 0, True: 1})
    

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

X_train_transformed = preprocessor.fit_transform(df_train)
feature_names_train = preprocessor.get_feature_names_out()
X_test_transformed = preprocessor.transform(df_test)
feature_names_test = preprocessor.get_feature_names_out()

In [26]:
# Преобразование X в numpy
if issparse(X_train_transformed):
    X_train_np = X_train_transformed.toarray()
    X_test_transformed = X_test_transformed.toarray()
else:
    X_train_np = X_train_transformed

# Учёт дисбаланса классов
pos_weight = 6.9 

In [43]:
# Гиперпараметры
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'num_leaves': 90,
    'max_depth': 8,
    'learning_rate': 0.1,
    'n_estimators': 200,
    'min_child_weight': 20,
    'lambda_l2': 0.3,
    'min_data_in_leaf': 50,
    'scale_pos_weight': pos_weight,
    'random_state': 42,
    'n_jobs': -1
}

# Обучение модели на всем тренировочном наборе
model = lgb.LGBMClassifier(**params)
model.fit(X_train_np, y_train_np)

# Предсказание вероятностей на тесте
y_proba_test = model.predict_proba(X_test_transformed)[:, 1]

# Подбор лучшего порога
thresholds = np.arange(0.75, 0.961, 0.01)
best_threshold = 0.75
best_metrics = {'precision': 0, 'recall': 0, 'f1': 0}

for threshold in thresholds:
    y_pred_test = (y_proba_test >= threshold).astype(int)
    precision = precision_score(y_test_np, y_pred_test, zero_division=0)
    recall = recall_score(y_test_np, y_pred_test)

    if recall >= 0.04 and precision > best_metrics['precision']:
        best_threshold = threshold
        best_metrics = {
            'precision': precision,
            'recall': recall,
            'f1': f1_score(y_test_np, y_pred_test)
        }

# Финальные метрики
y_pred_test = (y_proba_test >= best_threshold).astype(int)

print(f"\n🔍 Best threshold: {best_threshold:.2f}")
print(f"Precision: {best_metrics['precision']:.4f}")
print(f"Recall: {best_metrics['recall']:.4f}")
print(f"F1-score: {best_metrics['f1']:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_np, y_pred_test))



🔍 Best threshold: 0.95
Precision: 0.9310
Recall: 0.0449
F1-score: 0.0856

Confusion Matrix:
[[12448     6]
 [ 1725    81]]


In [29]:
undersampler = ClusterCentroids()
X_train_np_under, y_train_np_under = undersampler.fit_resample(X_train_np, y_train_np)

# Параметры модели
logit_params = {
    'penalty': 'l2',
    'solver': 'saga',
    'C': 0.1,
    'max_iter': 900,
    'class_weight': 'balanced',
    'random_state': 42,
    'n_jobs': -1
}

# Обучение модели
final_model = LogisticRegression(**logit_params)
final_model.fit(X_train_np_under, y_train_np_under)

# Предсказание вероятностей
y_proba = final_model.predict_proba(X_test_transformed)[:, 1]

# Подбор лучшего порога
thresholds = np.arange(0.5, 0.96, 0.01)
best_threshold = 0.5
best_metrics = {'precision': 0, 'recall': 0, 'f1': 0}

for threshold in thresholds:
    y_pred = (y_proba >= threshold).astype(int)
    precision = precision_score(y_test_np, y_pred, zero_division=0)
    recall = recall_score(y_test_np, y_pred)
    f1 = f1_score(y_test_np, y_pred)

    # Условие: минимальный recall и максимальная precision
    if recall >= 0.06 and precision > best_metrics['precision']:
        best_threshold = threshold
        best_metrics = {
            'precision': precision,
            'recall': recall,
            'f1': f1
        }

# Финальные предсказания с лучшим порогом
y_test_pred = (y_proba >= best_threshold).astype(int)

# Вывод результатов
print(f"\n🔍 Best threshold: {best_threshold:.2f}")
print(f"Precision: {best_metrics['precision']:.4f}")
print(f"Recall: {best_metrics['recall']:.4f}")
print(f"F1-score: {best_metrics['f1']:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_np, y_test_pred))


🔍 Best threshold: 0.77
Precision: 0.7176
Recall: 0.0676
F1-score: 0.1235

Confusion Matrix:
[[12406    48]
 [ 1684   122]]


In [46]:
# True Positives - TP
TP_lgbm_indices = np.where((y_pred_test == 1) & (y_test_np == 1))[0]
TP_logreg_indices = np.where((y_test_pred == 1) & (y_test_np == 1))[0]

# Разные TP (уникальные для каждой модели)
only_lgbm_TP = np.setdiff1d(TP_lgbm_indices, TP_logreg_indices)
only_logreg_TP = np.setdiff1d(TP_logreg_indices, TP_lgbm_indices)

# Все уникальные TP
all_unique_TP = np.union1d(TP_lgbm_indices, TP_logreg_indices)

# Вывод
print("TP только у LGBM:", len(only_lgbm_TP.tolist()))
print("TP только у Logreg:", len(only_logreg_TP.tolist()))
print("Общее количество уникальных TP:", len(all_unique_TP))

TP только у LGBM: 81
TP только у Logreg: 122
Общее количество уникальных TP: 203


In [47]:
# False Positives - FP
FP_lgbm_indices = np.where((y_pred_test == 1) & (y_test_np == 0))[0]
FP_logreg_indices = np.where((y_test_pred == 1) & (y_test_np == 0))[0]

# Разные FP (уникальные для каждой модели)
only_lgbm_FP = np.setdiff1d(FP_lgbm_indices, FP_logreg_indices)
only_logreg_FP = np.setdiff1d(FP_logreg_indices, FP_lgbm_indices)

# Все уникальные FP
all_unique_FP = np.union1d(FP_lgbm_indices, FP_logreg_indices)

# Вывод
print("TN только у LGBM:", len(only_lgbm_FP.tolist()))
print("TP только у Logreg:", len(only_logreg_FP.tolist()))
print("Общее количество уникальных TP:", len(all_unique_FP))

TN только у LGBM: 6
TP только у Logreg: 48
Общее количество уникальных TP: 54


In [48]:
#Вычисление precision и recall для модели hard voting-like

# Подсчет количества 0 и 1 в тестовой выборке
target_1_test = np.sum(y_test_np)  
target_0_test = len(y_test_np) - target_1_test  

print(f"Количество честных покупателей: {target_0_test}")
print(f"Количество мошенников: {target_1_test}")

# Итоговая точность и полнота
precision = len(all_unique_TP)/(len(all_unique_TP)+len(all_unique_FP))
recall = len(all_unique_TP)/target_1_test
print(f"Итоговая точность hard voting-like модели:{precision:.3f}")
print(f"Итоговая полнота hard voting-like модели:{recall:.3f}")

Количество честных покупателей: 12454
Количество мошенников: 1806
Итоговая точность hard voting-like модели:0.790
Итоговая полнота hard voting-like модели:0.112
